# LIGN

Graph Induced Lifelong Learning through Features Similarities and Dissimilarities 

----

## Imports

In [1]:
import lign as lg
import lign.models as md
import lign.utils as utl

import torch as th
import torchvision as tv
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler

import numpy as np
import datetime
tm_now = datetime.datetime.now

----

## Preprocessing 

### Create Dataset

In [ ]:
dataset_name = "CIFAR" #<<<<<

trans = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = utl.load.cifar_to_lign("data/datasets/CIFAR100", transforms = trans)
dataset.save("data/datasets/cifar100_train.lign")

validate = utl.load.cifar_to_lign("data/datasets/CIFAR100", train=False, transforms = trans)
validate.save("data/datasets/cifar100_test.lign")

In [ ]:
dataset_name = "MNIST" #<<<<<

trans = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.1307,), (0.3081,))
])

dataset = utl.load.mnist_to_lign("data/datasets/MNIST", transforms = trans)
dataset.save("data/datasets/mnist_train.lign")

validate = utl.load.mnist_to_lign("data/datasets/MNIST", train=False, transforms = trans)
validate.save("data/datasets/mnist_test.lign")

### Load Dataset

In [ ]:
dataset_name = "CIFAR" #<<<<<

dataset = lg.graph.GraphDataset("data/datasets/cifar100_train.lign")
validate = lg.graph.GraphDataset("data/datasets/cifar100_test.lign")

In [2]:
dataset_name = "MNIST" #<<<<<

dataset = lg.graph.GraphDataset("data/datasets/mnist_train.lign")
validate = lg.graph.GraphDataset("data/datasets/mnist_test.lign")

### Cuda GPUs

In [3]:
if th.cuda.is_available():
    device = th.device("cuda")
    th.cuda.empty_cache()
else:
    device = th.device("cpu")

### Functions

In [4]:
def sum_neighs_data(neighs): ## adds up neighbors' data before executing post_mod (pre_mod happens before)
    out = neighs[0]
    for neigh in neighs[1:]:
        out = out + neigh
    return out


### Hyperparameters
* LAMBDA: rate by which EPOCHS decreases over retraining periods
* DIST_VEC_SIZE: size of vector representing the mapping of the nodes by the model
* INIT_NUM_LAB: number of labels used to training the model initially in the supervised method to learn pairwise mapping
* LABELS: list of all the labels that model comes across. Labels can be appended at any time. The order of labels is initially randomized
* SUBGRAPH_SIZE: represent the number of nodes processed at once. The models don't have batches. This is the closest thing to it
* AMP_ENABLE: toggle to enable mixed precission training
* EPOCHS: Loops executed during training
* LR: Learning rate
* RETRAIN_PER: period between retraining based on number of labels seen. format: (offset, period)

In [14]:
LAMBDA = 0.05
DIST_VEC_SIZE = 200 #128
INIT_NUM_LAB = 50
LABELS = np.arange(100)
SUBGRPAH_SIZE = 500
AMP_ENABLE = True
EPOCHS = 300
LR = 1e-3
RETRAIN_PER = {
    "superv": (6, 3), # (offset, frequency)
    #"semi": (0, 15)
}

np.random.shuffle(LABELS)

---
## Models
### LIGN

[L]ifelong Learning [I]nduced by [G]raph [N]eural Networks Model (LIGN)

In [ ]:
### CIFAR
class LIGN_CIFAR(nn.Module):
    def __init__(self, out_feats):
        super(LIGN_CIFAR, self).__init__()
        self.gcn1 = lg.layers.GCN(nn.Conv2d(1, 32, 3, 1))
        self.gcn2 = lg.layers.GCN(nn.Conv2d(32, 64, 3, 1))
        self.gcn3 = lg.layers.GCN(nn.Linear(9216, out_feats))
        self.drop1 = nn.Dropout(0.25)
        self.drop2 = nn.Dropout(0.5)

    def forward(self, g, features, save=False):
        x = F.relu(self.gcn1(g, features))
        if(save):
            x_sv = x[:100]
            print(x_sv.shape)
            for i in range(x_sv.size(0)):
                save_image(th.unsqueeze(x_sv[i], 1), "data/thesis/mid/gcn1/img" + str(i) + ".png")

        x = F.relu(self.gcn2(g, x))
        if(save):
            x_sv = x[:100]
            for i in range(x_sv.size(0)):
                save_image(th.unsqueeze(x_sv[i], 1), "data/thesis/mid/gcn2/img" + str(i) + ".png")
        x = F.max_pool2d(x, 2)
        x = th.flatten(self.drop1(x), 1)
        x = F.relu(self.gcn3(g, x))
        if(save):
            x_sv = x[:100]
            save_image(th.unsqueeze(th.unsqueeze(x_sv, 1), -1), "data/thesis/mid/vector output.png", nrow=100)

        return self.drop2(x)

base = LIGN_CIFAR(DIST_VEC_SIZE).to(device)

In [6]:
### MNIST
class LIGN_MNIST(nn.Module):
    def __init__(self, out_feats):
        super(LIGN_MNIST, self).__init__()
        self.gcn1 = lg.layers.GCN(nn.Conv2d(1, 32, 3, 1))
        self.gcn2 = lg.layers.GCN(nn.Conv2d(32, 64, 3, 1))
        self.gcn3 = lg.layers.GCN(nn.Linear(9216, out_feats))
        self.drop1 = nn.Dropout(0.25)
        self.drop2 = nn.Dropout(0.5)

    def forward(self, g, features, save=False):
        x = F.relu(self.gcn1(g, features))
        if(save):
            x_sv = x[:100]
            print(x_sv.shape)
            for i in range(x_sv.size(0)):
                save_image(th.unsqueeze(x_sv[i], 1), "data/thesis/mid/gcn1/img" + str(i) + ".png")

        x = F.relu(self.gcn2(g, x))
        if(save):
            x_sv = x[:100]
            for i in range(x_sv.size(0)):
                save_image(th.unsqueeze(x_sv[i], 1), "data/thesis/mid/gcn2/img" + str(i) + ".png")
        x = F.max_pool2d(x, 2)
        x = th.flatten(self.drop1(x), 1)
        x = F.relu(self.gcn3(g, x))
        if(save):
            x_sv = x[:100]
            save_image(th.unsqueeze(th.unsqueeze(x_sv, 1), -1), "data/thesis/mid/vector output.png", nrow=100)

        return self.drop2(x)


class Classifier(nn.Module): ## temporality layer for training
    def __init__(self, in_fea, out_fea, device = 'cuda'):
        super(Classifier, self).__init__()
        self.addon = lg.layers.ADDON(in_fea, out_fea, device=device) # dynamic linear dense layer
    
    def forward(self, g, features):
        x = F.log_softmax(self.addon(g, features), dim=1)
        return x

base = LIGN_MNIST(DIST_VEC_SIZE).to(device) # base
classifier = Classifier(DIST_VEC_SIZE, INIT_NUM_LAB, device).to(device) # classifer

----
## Training
### Parameters

In [7]:
#opt
accuracy = []
log = []
num_of_labels = len(LABELS)

opt = th.optim.Adam([ # optimizer for the full
        {'params': base.parameters()},
        {'params': classifier.parameters(), 'lr': 1e-3}
    ], lr=1e-2)

scaler = GradScaler() if AMP_ENABLE else None

retrain_superv = lambda x: (x + RETRAIN_PER["superv"][0])%RETRAIN_PER["superv"][1] == 0
#retrain_semi = lambda x: (x + RETRAIN_PER["semi"][0])%RETRAIN_PER["semi"][1] == 0

### Load State

In [ ]:
checkpoint = th.load('data/models/LIGN_training_cool_time.pt')

base.load_state_dict(checkpoint['base'])
classifier.load_state_dict(checkpoint['classifier'])
opt.load_state_dict(checkpoint['optimizer'])

if AMP_ENABLE:
    scaler.load_state_dict(checkpoint['scaler'])

### Train Model

In [8]:
def test_and_save(num_labels, text, method="NN"):
    acc = lg.test.accuracy(base, validate, dataset, "x", "labels", LABELS[:num_labels], cluster=(utl.clustering.NN(), 5), device=device)
    accuracy.append(acc)
    log.append("Label: {}/{} -- Accuracy({}): {}% -- {}".format(num_labels, num_of_labels, method, round(acc, 2), text))
    print(log[-1])

# original network
test_and_save(INIT_NUM_LAB, "Original", method="NN")

lg.train.superv(base, classifier, opt, dataset, "x", "labels", LABELS[:INIT_NUM_LAB], (device, scaler), epochs=EPOCHS, subgraph_size=SUBGRPAH_SIZE)

# trained network
test_and_save(INIT_NUM_LAB, "Initial Training", method="NN")

# online learning system
for num_labels in range(INIT_NUM_LAB+1, num_of_labels+1):

    if retrain_superv(num_labels):
        test_and_save(num_labels, "Before Supervised Retraining", method="NN")

        addon.addon.update_size(num_labels)
        EPOCHS -= int(EPOCHS*LAMBDA)
        lg.train.superv(base, classifier, opt, dataset, "x", "labels", LABELS[:num_labels], (device, scaler), epochs=EPOCHS, subgraph_size=SUBGRPAH_SIZE)
    
    test_and_save(num_labels, "Test with " + str(LABELS[num_labels]), method="NN")


Label: 4/10 -- Accuracy: 86.07% -- Original
Label: 4/10 -- Accuracy: 98.86% -- Semisurpervised Retraining: False -- Surpervised Retraining: False
Label: 5/10 -- Accuracy: 92.02% -- Semisurpervised Retraining: False -- Surpervised Retraining: False
Label: 6/10 -- Accuracy: 97.91% -- Semisurpervised Retraining: False -- Surpervised Retraining: True
Label: 7/10 -- Accuracy: 91.1% -- Semisurpervised Retraining: False -- Surpervised Retraining: False
Label: 8/10 -- Accuracy: 83.92% -- Semisurpervised Retraining: False -- Surpervised Retraining: False
Label: 9/10 -- Accuracy: 95.35% -- Semisurpervised Retraining: False -- Surpervised Retraining: True
Label: 10/10 -- Accuracy: 93.88% -- Semisurpervised Retraining: False -- Surpervised Retraining: False


### Save State

In [9]:

time = str(tm_now()).replace(":", "-").replace(".", "").replace(" ", "_")
filename = "LIGN_" + dataset_name + "_training_"+time

## Save metrics
metrics = {
    "accuracy": accuracy,
    "log": log
}
utl.io.json(metrics, "data/metrics/"+filename+".json")

## Save hyperparameters
para = {
    "LAMBDA": LAMBDA,
    "DIST_VEC_SIZE": DIST_VEC_SIZE,
    "INIT_NUM_LAB": INIT_NUM_LAB,
    "LABELS": LABELS.tolist(),
    "SUBGRPAH_SIZE": SUBGRPAH_SIZE,
    "AMP_ENABLE": AMP_ENABLE,
    "EPOCHS": EPOCHS,
    "LR": LR,
    "RETRAIN_PER": RETRAIN_PER,
    "STRUCTURE": {
        "base": str(base),
        "classifier": str(classifier)
    }
}

utl.io.json(para, "data/parameters/"+filename+".json")

## Save model
check = {
    "base": base.state_dict(),
    "classifier": base.state_dict(),
    "optimizer": opt.state_dict()
}
if AMP_ENABLE:
    check["scaler"] = scaler.state_dict()

th.save(check, "data/models/"+filename+".pt")
    